Read and write to delta 

1) You can write to a managed delta tables (uses saveAsTable) or external location (uses save) 

In [0]:
import pandas as pd
import seaborn as sns
#
from pyspark.sql.functions import *
#
from databricks.feature_store import FeatureStoreClient, feature_table

In [0]:
taxis_df = sns.load_dataset("taxis")
#
taxis_sdf = spark.createDataFrame(taxis_df)
#
display(taxis_sdf.limit(5))

In [0]:
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .saveAsTable("taxis_sdf"))

In [0]:
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .save("/temp/taxis_sdf_external"))

list managed tables

In [0]:
display(spark.sql("show tables").limit(5))

In [0]:
taxis_sdf    = spark.table("taxis_sdf")           # read managed delta table


In [0]:
taxi_sdf_ext = spark.read.load("/temp/taxis_sdf_external") # read external delta table

In [0]:
display(taxi_sdf_ext.head(10))

View Delta Table version history        

In [0]:
# remove nulls for payment column
taxis_sdf = taxis_sdf.filter("payment is not null")
#
# save the change to managed delta table
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .saveAsTable("taxis_sdf"))
#
# save the change to external delta table
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .save("/temp/taxis_sdf_external"))

See changes in managed tables 

In [0]:
%sql
DESCRIBE HISTORY taxis_sdf

In [0]:
%sql
DESCRIBE HISTORY '/temp/taxis_sdf_external/'

In [0]:
display(spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .table("taxis_sdf")
        .filter("payment is null")
        .limit(5))
#
# Or for an external delta table
display(spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load("/temp/taxis_sdf")
        .filter("payment is null")
        .limit(5))

Feature store 


fs.create_table , Either provide df or shcmea

fs.write_table mode = overwrit or merge 

fs.read_table

fs.drop_table